In [14]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [7]:
print("Hello")

Hello


In [8]:
# !pip install transformers torch accelerate bitsandbytes

In [9]:
# !pip install huggingface_hub

In [10]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

In [11]:

# Load environment variables from .env
load_dotenv()

# Get the Hugging Face Token
token = os.getenv("HUGGINGFACE_TOKEN")

# Use the token in your script
print(f"Hugging Face Token: {token[:4]}********")  # Mask for security
login(token=access_token)

Hugging Face Token: hf_B********


In [12]:
login(token=access_token)

NameError: name 'access_token' is not defined

In [13]:
# =========================================
# 1) Imports
# =========================================
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [14]:
from datasets import Dataset

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Load the dataset
csv_path = "/home/cse/Desktop/VOIP_VISHING/REPOS/conversation-prediction/FINAL_DATASET2.csv"
df = pd.read_csv(csv_path)

# Group by CONVERSATION_ID and create full conversations
df['FULL_CONVERSATION'] = df.groupby(['CONVERSATION_ID'])['TEXT'].transform(lambda x: ' '.join(x))

# Create partial conversations (excluding the last step)
df['PARTIAL_CONVERSATION'] = df.groupby(['CONVERSATION_ID'])['TEXT'].transform(lambda x: ' '.join(x[:-1]))

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1")

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['PARTIAL_CONVERSATION'], padding="max_length", truncation=True, max_length=512)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/870 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load the model
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")

In [18]:
# Move the model to GPU
model = model.to("cuda")

# Test inference
input_text = "Hello?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate output
outputs = model.generate(**inputs, max_length=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Hello? I'm trying to find a way to solve this problem: Given a sequence of n


In [19]:
import torch
torch.cuda.empty_cache()

In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,  # Gradient accumulation
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=10_000,
    logging_dir='./logs',
    logging_steps=200,
    fp16=True,  # Mixed precision training
)


# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,  # Your tokenized training dataset
    eval_dataset=tokenized_val,     # Your tokenized validation dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_deepseek_distill")
tokenizer.save_pretrained("./fine_tuned_deepseek_distill")

/home/cse/anaconda3/envs/voipvishing/lib/python3.13/site-packages/transformers/training_args.py:1612: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.